## BGNBD & GG ile CLTV Tahmini

İş Problemi:
    
Bir e-ticaret sitesi müşteri aksiyonları için
müşterilerinin CLTV değerlerine göre ileriye
dönük bir projeksiyon yapılmasını istemektedir.
Elinizdeki veriseti ile 1 aylık yada 6 aylık zaman
periyotları içerisinde en çok gelir getirebilecek
müşterileri tespit etmek mümkün müdür?.
    
    

Veri Seti Hikayesi:
    
    Online Retail II https://archive.ics.uci.edu/ml/datasets/Online+Retail+II
isimli veri seti İngiltere merkezli online bir satış
mağazasının 01/12/2009 09/12/2011 tarihleri arasındaki satışlarını
içeriyor.
Bu şirketin ürün kataloğunda hediyelik eşyalar yer alıyor. Promosyon
ürünleri olarak da düşünülebilir.
Çoğu müşterisinin toptancı olduğu bilgisi de mevcut.

In [3]:
import pandas as pd
import datetime as dt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from sklearn.preprocessing import MinMaxScaler

In [4]:
desired_width = 300
pd.set_option('display.width', desired_width)
pd.pandas.set_option('display.max_columns', None)

In [5]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [6]:
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")
df = df_.copy()

### Veri Ön İşleme

In [7]:
df.describe().T
df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]

replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397885.0,11.830770,25.523052,1.000,2.00,6.00,12.00,298.50
Price,397885.0,2.893492,3.227175,0.001,1.25,1.95,3.75,37.06
Customer ID,397885.0,15294.416882,1713.144421,12346.000,13969.00,15159.00,16795.00,18287.00


In [10]:
df["TotalPrice"] = df["Quantity"] * df["Price"]
df["Country"].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Finland',
       'Austria', 'Greece', 'Singapore', 'Lebanon',
       'United Arab Emirates', 'Israel', 'Saudi Arabia', 'Czech Republic',
       'Canada', 'Unspecified', 'Brazil', 'USA', 'European Community',
       'Bahrain', 'Malta', 'RSA'], dtype=object)

In [16]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6.0,2011-12-09 12:50:00,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4.0,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4.0,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3.0,2011-12-09 12:50:00,4.95,12680.0,France,14.85


In [20]:
today_date = dt.datetime(2011, 12, 11)

def create_cltv_p(dataframe, month=3, info = True,Country = "ALL"):
    if Country == "ALL":
        cltv_df = dataframe.groupby('Customer ID').agg(
            {'InvoiceDate': [lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days,
                             lambda InvoiceDate: (today_date - InvoiceDate.min()).days],
             'Invoice': lambda Invoice: Invoice.nunique(),
             'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
    else:
         cltv_df = dataframe[dataframe["Country"] == Country].groupby('Customer ID').agg(
            {'InvoiceDate': [lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days,
                             lambda InvoiceDate: (today_date - InvoiceDate.min()).days],
            'Invoice': lambda Invoice: Invoice.nunique(),
            'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

    cltv_df.columns = cltv_df.columns.droplevel(0)
    cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']
    cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]
    cltv_df = cltv_df[(cltv_df['frequency'] > 1)]
    cltv_df["recency"] = cltv_df["recency"] / 7
    cltv_df["T"] = cltv_df["T"] / 7

    # 2. BG-NBD Modelinin Kurulması
    bgf = BetaGeoFitter(penalizer_coef=0.001)
    bgf.fit(cltv_df['frequency'],
            cltv_df['recency'],
            cltv_df['T'])
    if Country == "ALL":
      cltv_df["expected_purc_1_week"] = bgf.predict(1,
                                                  cltv_df['frequency'],
                                                  cltv_df['recency'],
                                                  cltv_df['T'])

      cltv_df["expected_purc_1_month"] = bgf.predict(4,
                                                   cltv_df['frequency'],
                                                   cltv_df['recency'],
                                                   cltv_df['T'])

    # 3. GAMMA-GAMMA Modelinin Kurulması
    ggf = GammaGammaFitter(penalizer_coef=0.01)
    ggf.fit(cltv_df['frequency'], cltv_df['monetary'])
    cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                                 cltv_df['monetary'])

    # 4. BG-NBD ve GG modeli ile CLTV'nin hesaplanması.
    cltv = ggf.customer_lifetime_value(bgf,
                                       cltv_df['frequency'],
                                       cltv_df['recency'],
                                       cltv_df['T'],
                                       cltv_df['monetary'],
                                       time=month,  # 3 aylık
                                       freq="W",  # T'nin frekans bilgisi.
                                       discount_rate=0.01)

    cltv = cltv.reset_index()
    cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")
    #cltv_final["segment"] = pd.qcut(cltv_final["clv"], 4, labels=["D", "C", "B", "A"])

    if info:
        return cltv_final.rename(columns = {'clv':"clv_"+str(month)+"_Ay"})
    else:
        return cltv_final["clv"]

In [22]:
cltv_UK = create_cltv_p(df,month = 6,info = True,Country = "United Kingdom")
cltv_UK.sort_values(by="clv_6_Ay", ascending=False).head(10)
## standartlaştırma
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(cltv_UK[["clv_6_Ay"]])
cltv_UK["scaled_clv_6ay"] = scaler.transform(cltv_UK[["clv_6_Ay"]])
cltv_UK.sort_values(by="scaled_clv_6ay", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_average_profit,clv_6_Ay,scaled_clv_6ay
2486,18102.0,52.285714,52.571429,60,3859.739083,3870.838975,92215.233584,1.000000
589,14096.0,13.857143,14.571429,17,3163.588235,3195.972563,55726.778046,0.604312
2184,17450.0,51.285714,52.571429,46,2863.274891,2874.045685,52846.107624,0.573074
2213,17511.0,52.857143,53.428571,31,2933.943065,2950.346629,36947.207911,0.400663
1804,16684.0,50.428571,51.285714,28,2209.969107,2223.690179,26146.294267,0.283536
406,13694.0,52.714286,53.428571,50,1275.700500,1280.155796,25224.934931,0.273544
587,14088.0,44.571429,46.142857,13,3864.554615,3916.384282,25041.839323,0.271559
1485,16000.0,0.000000,0.428571,3,2335.120000,2477.667281,23975.522233,0.259995
1173,15311.0,53.285714,53.428571,91,667.779121,669.078222,23597.164785,0.255892
133,13089.0,52.285714,52.857143,97,606.362474,607.472561,22971.583929,0.249108


In [23]:
cltv_UK[["clv_6_Ay","scaled_clv_6ay"]].describe().T

,count,mean,std,min,25%,50%,75%,max
clv_6_Ay,2570.0,1528.111703,3156.865358,9.316331e-22,498.481119,954.561825,1705.695386,92215.233584
scaled_clv_6ay,2570.0,0.016571,0.034234,0.000000e+00,0.005406,0.010351,0.018497,1.000000


United Kingdom için :
    
İçin 6 ay da en çok gelir getirebilecek ilk 10 müşteriyi scaled edilmiş veriye göre sıraladığımızda
Önem düzeyi yüksek olan müşterileri görebiliryoruz. diğer kalan 6 müşteri ise benzer aralıkllarda bir dağılım göstermiş.
CustomerID  scaled_clv_6ay
18102       1.000000
14096       0.604312
17450       0.573074
17511       0.400663

scaled_clv_6ay  describe özelliklerine baktığımızda ise ortalama olarak 0.016571 gözüküyor. 
Std'si ise 0.034234

In [25]:
cltv_UK = create_cltv_p(df,month = 1,info = True,Country = "United Kingdom") #1 ay
cltv_UK["clv_12_Ay"] = create_cltv_p(df,month = 12,info = False,Country = "United Kingdom") #12 ay

## standartlaştırma
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(cltv_UK[["clv_1_Ay"]])
cltv_UK["scaled_clv_1ay"] = scaler.transform(cltv_UK[["clv_1_Ay"]])

scaler.fit(cltv_UK[["clv_12_Ay"]])
cltv_UK["scaled_clv_12ay"] = scaler.transform(cltv_UK[["clv_12_Ay"]])

cltv_UK.sort_values(by="clv_1_Ay", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_average_profit,clv_1_Ay,clv_12_Ay,scaled_clv_1ay,scaled_clv_12ay
2486,18102.0,52.285714,52.571429,60,3859.739083,3870.838975,16025.706602,176129.037489,1.000000,1.000000
589,14096.0,13.857143,14.571429,17,3163.588235,3195.972563,9869.301396,105044.444809,0.615842,0.596406
2184,17450.0,51.285714,52.571429,46,2863.274891,2874.045685,9184.276955,100931.571501,0.573097,0.573055
2213,17511.0,52.857143,53.428571,31,2933.943065,2950.346629,6420.409512,70572.713868,0.400632,0.400688
1804,16684.0,50.428571,51.285714,28,2209.969107,2223.690179,4545.907460,49920.554241,0.283663,0.283432
406,13694.0,52.714286,53.428571,50,1275.700500,1280.155796,4382.993041,48185.724453,0.273498,0.273582
1485,16000.0,0.000000,0.428571,3,2335.120000,2477.667281,4365.549314,44563.162131,0.272409,0.253014
587,14088.0,44.571429,46.142857,13,3864.554615,3916.384282,4361.008315,47749.578501,0.272126,0.271106
1173,15311.0,53.285714,53.428571,91,667.779121,669.078222,4099.862530,45078.949580,0.255830,0.255943
133,13089.0,52.285714,52.857143,97,606.362474,607.472561,3991.668085,43879.416847,0.249079,0.249132


In [26]:
cltv_UK.sort_values(by="clv_12_Ay", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_average_profit,clv_1_Ay,clv_12_Ay,scaled_clv_1ay,scaled_clv_12ay
2486,18102.0,52.285714,52.571429,60,3859.739083,3870.838975,16025.706602,176129.037489,1.000000,1.000000
589,14096.0,13.857143,14.571429,17,3163.588235,3195.972563,9869.301396,105044.444809,0.615842,0.596406
2184,17450.0,51.285714,52.571429,46,2863.274891,2874.045685,9184.276955,100931.571501,0.573097,0.573055
2213,17511.0,52.857143,53.428571,31,2933.943065,2950.346629,6420.409512,70572.713868,0.400632,0.400688
1804,16684.0,50.428571,51.285714,28,2209.969107,2223.690179,4545.907460,49920.554241,0.283663,0.283432
406,13694.0,52.714286,53.428571,50,1275.700500,1280.155796,4382.993041,48185.724453,0.273498,0.273582
587,14088.0,44.571429,46.142857,13,3864.554615,3916.384282,4361.008315,47749.578501,0.272126,0.271106
1173,15311.0,53.285714,53.428571,91,667.779121,669.078222,4099.862530,45078.949580,0.255830,0.255943
1485,16000.0,0.000000,0.428571,3,2335.120000,2477.667281,4365.549314,44563.162131,0.272409,0.253014
133,13089.0,52.285714,52.857143,97,606.362474,607.472561,3991.668085,43879.416847,0.249079,0.249132


Çok fazla fark yok ilk 6 kişi hiç değişmemiş bunun nedeni ilk ayda o müşterinin yapacağı cltv skoru ile 12 aylık ClTV
skoru o ilk aya göre katlanarak gideceğinden dolayı sıralama çok değişmez

fakat birbirine yakın olan cltv değerleri için bunu tam olarak diyemeyiz.

CustomerID  16000.0 olan müşteri 1. ayda 7. sıradayken.
            12aylık cltv hesaplandığında  9. sıraya geriliyor

çünkü dierlerine göre cltv skoru düşüyor.            

In [27]:
cltv_UK = create_cltv_p(df,month = 6,info = True,Country = "United Kingdom") #6 ay
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(cltv_UK[["clv_6_Ay"]])
cltv_UK["scaled_clv_6ay"] = scaler.transform(cltv_UK[["clv_6_Ay"]])

cltv_UK = cltv_UK.reset_index()
cltv_UK["segment"] = pd.qcut(cltv_UK["clv_6_Ay"], 4, labels=["D", "C", "B", "A"])
cltv_UK = pd.DataFrame(cltv_UK)


cltv_UK[["segment","recency","frequency","clv_6_Ay","scaled_clv_6ay"]].groupby("segment").agg({"mean","sum"})

recency            frequency                 clv_6_Ay              scaled_clv_6ay          
                  sum       mean       sum       mean           sum         mean            sum      mean
segment                                                                                                  
D        14191.000000  22.069984      1973   3.068429  1.739652e+05   270.552438       1.886513  0.002934
C        19795.142857  30.833556      2561   3.989097  4.574626e+05   712.558529       4.960814  0.007727
B        19166.285714  29.854028      3504   5.457944  8.180803e+05  1274.268363       8.871422  0.013818
A        20219.428571  31.445457      7258  11.287714  2.477739e+06  3853.404346      26.869085  0.041787

Burada incelendiğinde en iyi A GURUBU EN kötü ise D gurubu olarak gözüküyor.
A gurubunu premium müşteri olarak belirlenip iş gücünü buraya yönlendirilmeli.
D gurubunda ise ortalama alama sıklığı 3 olarak gözüküyor neden bu müşterilerin önüne daha fazla 
alabilieceği ürünler gösterilmeli.